VacationPy

Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zhezqazghan,47.7833,67.7667,24.42,26,0,2.13,KZ,1722719043
1,1,vorgashor,67.5833,63.9500,12.68,67,100,7.13,RU,1722719044
2,2,tashtagol,52.7657,87.8894,15.26,97,100,0.67,RU,1722719045
3,3,puerto natales,-51.7236,-72.4875,3.25,93,75,1.03,CL,1722719047
4,4,georgetown,5.4112,100.3354,25.96,74,20,1.54,MY,1722719048


Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])
}

coordinates_df = pd.DataFrame(coordinates)

# Configure the map plot_4
humidity_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)
   

# Display the map plot
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21, #Wind speed less than 4 m/s, #Zero cloudiness
weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 20) & \
                                (city_data_df["Wind Speed"] < 4) & \
                                (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
weather_df = weather_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zhezqazghan,47.7833,67.7667,24.42,26,0,2.13,KZ,1722719043
81,81,correntina,-13.3433,-44.6367,23.96,30,0,2.28,BR,1722719134
111,111,smithers,54.7804,-127.1743,25.18,23,0,1.54,CA,1722719166
158,158,sarykemer,43.0000,71.5000,22.71,30,0,1.74,KZ,1722719217
174,174,belmonte,-15.8631,-38.8828,22.49,84,0,3.46,BR,1722719234
214,214,tirhanimine,35.2362,-3.9545,24.39,60,0,1.47,MA,1722719278
233,233,changyon,38.2508,125.0961,24.70,92,0,1.01,KP,1722719300
290,290,boysun,38.2061,67.1986,25.64,23,0,2.57,UZ,1722719360
407,407,javanrud,34.7961,46.5172,24.38,28,0,3.23,IR,1722719490
408,408,moyynkum,44.2914,72.9469,26.73,26,0,1.51,KZ,1722719492


Step 3: Create a new DataFrame called hotel_df.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,zhezqazghan,KZ,47.7833,67.7667,26,
81,correntina,BR,-13.3433,-44.6367,30,
111,smithers,CA,54.7804,-127.1743,23,
158,sarykemer,KZ,43.0000,71.5000,30,
174,belmonte,BR,-15.8631,-38.8828,84,
214,tirhanimine,MA,35.2362,-3.9545,60,
233,changyon,KP,38.2508,125.0961,92,
290,boysun,UZ,38.2061,67.1986,23,
407,javanrud,IR,34.7961,46.5172,28,
408,moyynkum,KZ,44.2914,72.9469,26,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {
    "categories": ",".join(categories),
    "apiKey": geoapify_key
}
 
# Print a message to follow up the hotel search
print("Starting hotel search")
 
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
 
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
 
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
 
    try:
        # Make an API request using the params dictionary
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
 
        # Convert the API response to JSON format
        name_address = response.json()
 
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (requests.RequestException, KeyError, IndexError) as e:
        # If no hotel is found or there's an error, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(f"Error occurred for {row['City']}: {str(e)}")
 
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
 
# Display sample data
hotel_df


Starting hotel search
Error occurred for zhezqazghan: HTTPSConnectionPool(host='api.geoapify.com', port=443): Max retries exceeded with url: /v2/places?categories=accommodation.hotel&apiKey=b563d58be8e24acd8e85c319b7c4947c&filter=circle%3A67.7667%2C47.7833%2C10000&bias=proximity%3A67.7667%2C47.7833 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022FA4993CD0>: Failed to resolve 'api.geoapify.com' ([Errno 11001] getaddrinfo failed)"))
zhezqazghan - nearest hotel: No hotel found
correntina - nearest hotel: Pousada Silva
smithers - nearest hotel: Sunshine Inn Hotel
Error occurred for sarykemer: list index out of range
sarykemer - nearest hotel: No hotel found
Error occurred for belmonte: list index out of range
belmonte - nearest hotel: No hotel found
tirhanimine - nearest hotel: Mira Palace
Error occurred for changyon: list index out of range
changyon - nearest hotel: No hotel found
boysun - nearest hotel: Grand Hotel
Error occurred for javanrud: list 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,zhezqazghan,KZ,47.7833,67.7667,26,No hotel found
81,correntina,BR,-13.3433,-44.6367,30,Pousada Silva
111,smithers,CA,54.7804,-127.1743,23,Sunshine Inn Hotel
158,sarykemer,KZ,43.0000,71.5000,30,No hotel found
174,belmonte,BR,-15.8631,-38.8828,84,No hotel found
214,tirhanimine,MA,35.2362,-3.9545,60,Mira Palace
233,changyon,KP,38.2508,125.0961,92,No hotel found
290,boysun,UZ,38.2061,67.1986,23,Grand Hotel
407,javanrud,IR,34.7961,46.5172,28,No hotel found
408,moyynkum,KZ,44.2914,72.9469,26,No hotel found


Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display


# Configure the map plot_4
city_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ['Hotel Name', 'Country']
)

city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)